<a href="https://colab.research.google.com/github/Panperception/QKD/blob/main/QKD_AI3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required libs

In [ ]:
# %pip uninstall qiskit
!pip install pyqmc
!pip install qiskit
!pip install qiskit-aer
!pip install qiskit-algorithms
!pip install qiskit-nature
!pip install qutip
!pip install ase
!pip install scipy
!nvcc --version

## Initialization

In [ ]:
# TBC

#### Data Preparation

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Generate dataset
def generate_qkd_dataset(samples=1000):
    data = []
    for _ in range(samples):
        # Simulate normal transmission
        alice_bits, alice_bases = generate_bb84_bits(100)
        qc_list = prepare_qubits(alice_bits, alice_bases)
        bob_bases = np.random.randint(0, 2, 100)
        bob_measurements = measure_qubits(qc_list, bob_bases)

        # Simulate attack/noise
        if np.random.rand() < 0.5:  # 50% chance of attack
            bob_measurements = simulate_noisy_bb84(qc_list, bob_bases)
            label = 1  # Attack
        else:
            label = 0  # No Attack

        qber = np.sum(np.array(alice_bits) != np.array(bob_measurements)) / 100
        data.append([qber, label])

    return pd.DataFrame(data, columns=['QBER', 'Attack'])

df = generate_qkd_dataset()
X_train, X_test, y_train, y_test = train_test_split(df[['QBER']], df['Attack'], test_size=0.2)


## 1. Real‐Time Dynamic Key Rate Optimization via Reinforcement Learning

In this example, we create a custom Gym environment that simulates a QKD channel (with state variables like channel loss and noise). An RL agent (using DQN from stable‐baselines3) learns to adjust a modulation parameter in real time to maximize the secure key rate.

In [8]:
!pip install stable-baselines3
!pip install 'shimmy>=2.0'
import gym
from gym import spaces
import numpy as np
from stable_baselines3 import DQN

class QKDEnv(gym.Env):
    def __init__(self):
        super(QKDEnv, self).__init__()
        # State: [channel_loss, noise_level] (both between 0 and 1)
        self.observation_space = spaces.Box(low=0, high=1, shape=(2,), dtype=np.float32)
        # Action: continuous modulation parameter in [0,1]
        self.action_space = spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)
        self.state = np.array([0.5, 0.5])
        self.step_count = 0

    def step(self, action):
        channel_loss, noise = self.state
        modulation = action[0]
        # Simulate QBER: lower if modulation is optimal
        qber = noise * (1 - modulation) + np.random.rand()*0.01
        # Key rate improves when QBER is low and modulation is high
        key_rate = modulation * (1 - qber)
        reward = key_rate  # our goal is to maximize the key rate
        # Update state with slight random fluctuations
        self.state = np.clip(self.state + np.random.randn(2)*0.01, 0, 1)
        self.step_count += 1
        done = self.step_count > 100
        return self.state, reward, done, {}

    def reset(self):
        self.state = np.array([0.5, 0.5])
        self.step_count = 0
        return self.state

# Create and train the RL agent
env = QKDEnv()
model = DQN('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


AssertionError: The algorithm only supports (<class 'gymnasium.spaces.discrete.Discrete'>,) as action spaces but Box(0.0, 1.0, (1,), float32) was provided

## 2. Adaptive Error Correction Parameter Estimation
Here, we train a simple feed-forward neural network to predict an optimal error-correction code rate from channel parameters (e.g. channel loss, noise, QBER, etc.). This could then be used to adjust post-processing in QKD.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Generate synthetic training data: features [channel_loss, noise, QBER, misc_stat]
X = np.random.rand(1000, 4)
# Simulated optimal error correction rate: lower QBER yields higher optimal rate
y = 1 - X[:, 2] + 0.05*np.random.randn(1000)

model_ec = Sequential([
    Dense(64, activation='relu', input_shape=(4,)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])
model_ec.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_ec.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 0.4892 - mae: 0.6119 - val_loss: 0.1142 - val_mae: 0.2803
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1025 - mae: 0.2700 - val_loss: 0.0554 - val_mae: 0.1930
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 0.0713 - mae: 0.2157 - val_loss: 0.0343 - val_mae: 0.1481
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0443 - mae: 0.1720 - val_loss: 0.0238 - val_mae: 0.1221
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0315 - mae: 0.1409 - val_loss: 0.0180 - val_mae: 0.1058
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - loss: 0.0257 - mae: 0.1268 - val_loss: 0.0140 - val_mae: 0.0941
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0230 - mae: 0.1197 - val_loss: 0.0119 - val_mae: 0.0880
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - loss: 0.0177 - mae: 0.1056 - val_loss: 0.0095 - val_mae: 0.0779
Epoch 9/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.017

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Generate synthetic training data: features [channel_loss, noise, QBER, misc_stat]
X = np.random.rand(1000, 4)
# Simulated optimal error correction rate: lower QBER yields higher optimal rate
y = 1 - X[:, 2] + 0.05*np.random.randn(1000)

model_ec = Sequential([
    Dense(64, activation='relu', input_shape=(4,)),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])
model_ec.compile(optimizer='adam', loss='mse', metrics=['mae'])
model_ec.fit(X, y, epochs=50, batch_size=32, validation_split=0.2)


#### 3. Quantum Feedback Control Using an LSTM
This snippet simulates a feedback loop where past QBER measurements are fed into an LSTM model that predicts a corrective control signal. In practice, such a model could trigger adjustments in the QKD hardware to counteract disturbances.


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

def generate_qber_series(num_series=500, timesteps=20):
    X = []
    y = []
    for _ in range(num_series):
        base = np.random.rand() * 0.1  # baseline QBER
        noise = np.random.randn(timesteps) * 0.01
        series = base + noise
        # Control signal: aim to reduce QBER toward a target (e.g., 0.02)
        target = 0.02
        control = -(series[-1] - target)
        X.append(series.reshape(timesteps, 1))
        y.append(control)
    return np.array(X), np.array(y)

X_qber, y_feedback = generate_qber_series()
model_feedback = Sequential([
    LSTM(32, input_shape=(X_qber.shape[1], 1)),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])
model_feedback.compile(optimizer='adam', loss='mse')
model_feedback.fit(X_qber, y_feedback, epochs=20, batch_size=32, validation_split=0.2)


Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - loss: 0.0015 - val_loss: 4.1732e-04
Epoch 2/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 4.4647e-04 - val_loss: 2.2894e-04
Epoch 3/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.7026e-04 - val_loss: 1.0439e-04
Epoch 4/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 9.9974e-05 - val_loss: 9.4952e-05
Epoch 5/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 9.0602e-05 - val_loss: 1.0008e-04
Epoch 6/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 8.9565e-05 - val_loss: 9.3888e-05
Epoch 7/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 8.3520e-05 - val_loss: 8.9395e-05
Epoch 8/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 8.3796e-05 - val_loss: 1.2089e-04
Epoch 9/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 9.0998e-05 - val_loss: 8.4587e-05
Epoch 10/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 8.7409e-05 - val_loss: 8.2109e-05
Epoch 11/20
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 8.9452e-05 - val_loss: 8

#### 4. Adaptive Decoy-State Protocol Optimization
Here, we illustrate two approaches. First, a simple grid-search simulation that computes the secure key rate as a function of decoy intensity under a fixed channel loss. Second, a neural network model that learns to predict the optimal decoy intensity given channel conditions.

(a) Grid Search Approach:

In [ ]:
import numpy as np

def simulate_key_rate(decoy_intensity, channel_loss):
    # Simple simulation: key_rate maximized near decoy_intensity = 0.5
    penalty = 0.1
    key_rate = decoy_intensity * (1 - channel_loss) - penalty * abs(decoy_intensity - 0.5)
    return key_rate

channel_loss = 0.3
intensities = np.linspace(0.1, 1.0, 50)
key_rates = [simulate_key_rate(I, channel_loss) for I in intensities]
optimal_intensity = intensities[np.argmax(key_rates)]
print("Optimal decoy intensity (grid search):", optimal_intensity)


(b) Neural Network Predictor:

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Synthetic training data: features [channel_loss, noise_level]; label: optimal decoy intensity
X_train_decoy = np.random.rand(1000, 2)
# Simulate optimal decoy intensity as a function of channel_loss (for example purposes)
y_train_decoy = 0.5 + 0.2*(X_train_decoy[:,0] - 0.5) + 0.05*np.random.randn(1000)

model_decoy = Sequential([
    Dense(32, activation='relu', input_shape=(2,)),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])
model_decoy.compile(optimizer='adam', loss='mse')
model_decoy.fit(X_train_decoy, y_train_decoy, epochs=30, batch_size=32, validation_split=0.2)


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0561 - val_loss: 0.0141
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0139 - val_loss: 0.0110
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0105 - val_loss: 0.0067
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0063 - val_loss: 0.0044
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0041 - val_loss: 0.0031
Epoch 6/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0029 - val_loss: 0.0026
Epoch 7/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 8/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0026 - val_loss: 0.0024
Epoch 9/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0027 - val_loss: 0.0024
Epoch 10/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0025 - val_loss: 0.0024
Epoch 11/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 12/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0026 - val_l

#### 5. Multi-Attack and Side-Channel Detection via Multi-Modal Fusion
This example builds a multi-input neural network that fuses different sensor signals—such as QBER readings, detector noise statistics, and timing jitter—to classify whether the system is under attack.

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout

# Assume three inputs:
#   - QBER features (5 values)
#   - Detector noise stats (3 values)
#   - Timing jitter (2 values)
input_qber = Input(shape=(5,))
input_noise = Input(shape=(3,))
input_jitter = Input(shape=(2,))

x1 = Dense(16, activation='relu')(input_qber)
x2 = Dense(16, activation='relu')(input_noise)
x3 = Dense(16, activation='relu')(input_jitter)

combined = Concatenate()([x1, x2, x3])
x = Dense(32, activation='relu')(combined)
x = Dropout(0.2)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification: 0 (normal), 1 (attack)

model_multi = Model(inputs=[input_qber, input_noise, input_jitter], outputs=output)
model_multi.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_multi.summary()

# Generate synthetic data
num_samples = 1000
X_qber = np.random.rand(num_samples, 5)
X_noise = np.random.rand(num_samples, 3)
X_jitter = np.random.rand(num_samples, 2)
y_labels = np.random.randint(0, 2, size=(num_samples, 1))
model_multi.fit([X_qber, X_noise, X_jitter], y_labels, epochs=20, batch_size=32, validation_split=0.2)


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 5)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_4             │ (None, 3)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 16)             │             96 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 16)             │             64 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 16)             │             48 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 48)             │              0 │ dense_8[0][0],         │
│                           │                        │                │ dense_9[0][0],         │
│                           │                        │                │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 32)             │          1,568 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 32)             │              0 │ dense_11[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 1)              │             33 │ dropout_1[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,809 (7.07 KB)

 Trainable params: 1,809 (7.07 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.4643 - loss: 0.7009 - val_accuracy: 0.4900 - val_loss: 0.6940
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5262 - loss: 0.6930 - val_accuracy: 0.5000 - val_loss: 0.6938
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5072 - loss: 0.6929 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5007 - loss: 0.6910 - val_accuracy: 0.5000 - val_loss: 0.6939
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5500 - loss: 0.6874 - val_accuracy: 0.5200 - val_loss: 0.6937
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5329 - loss: 0.6911 - val_accuracy: 0.5100 - val_loss: 0.6947
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5387 - loss: 0.6882 - val_accuracy: 0.5150 - val_loss: 0.6942
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5478 - loss: 0.6886 - val_accuracy: 0.5250 - val_loss

#### 6. End-to-End QKD Protocol Design via Meta-Learning (Genetic Algorithm)
Here, we use a simple genetic algorithm (using the DEAP library) to optimize protocol parameters (e.g., modulation index, decoy intensity, error-correction rate) with the goal of maximizing a simulated secure key rate.

In [6]:
!pip install deap
import random
from deap import base, creator, tools, algorithms

# Define a simple evaluation function for protocol parameters
def evaluate_protocol(individual):
    # individual: [modulation_index, decoy_intensity, error_corr_rate]
    modulation_index, decoy_intensity, error_corr_rate = individual
    # Simulate key rate: maximize when decoy_intensity is near 0.5 and modulation and error_corr_rate are high
    key_rate = modulation_index * (1 - abs(decoy_intensity - 0.5)) * error_corr_rate
    return (key_rate,)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
# Parameters in [0,1]
toolbox.register("attr_float", random.uniform, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, 3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate_protocol)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=50)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=20, verbose=True)
best_ind = tools.selBest(population, 1)[0]
print("Best protocol parameters:", best_ind, "with key rate:", evaluate_protocol(best_ind)[0])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.9 MB/s eta 0:00:00
gen	nevals
0  	50    
1  	33    
2  	37    
3  	32    
4  	26    
5  	30    
6  	33    
7  	31    
8  	31    
9  	31    
10 	39    
11 	33    
12 	28    
13 	31    
14 	31    
15 	31    
16 	26    
17 	29    
18 	37    
19 	31    
20 	30    
Best protocol parameters: [1.6571893733611618, 0.47798639631153417, 1.614564446365225] with key rate: 2.6167385856144985
